# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.19it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:02<00:02,  1.18s/it]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:03<00:01,  1.39s/it]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:04<00:00,  1.19s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:04<00:00,  1.15s/it]



...

 78%|███████▊  | 18/23 [00:04<00:01,  4.86it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  5.02it/s]

 91%|█████████▏| 21/23 [00:05<00:00,  5.08it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  4.94it/s]

100%|██████████| 23/23 [00:05<00:00,  4.12it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Ashley and I'm a 31-year-old aspiring food blogger and amateur chef. I'm excited to start this new chapter of my life and share my passion for food with you all. On my blog, you'll find a variety of recipes, cooking tips, and stories about my culinary adventures. I'm still learning, but I'm excited to share my journey with you and hopefully inspire you to get into the kitchen and start cooking!
In this post, I'll be sharing my take on a classic recipe for chicken parmesan, a dish that I've been obsessed with since I first tried it at a family Italian restaurant. There's
Prompt: The president of the United States is
Generated text:  trying to get an agreement signed by Israel and the Palestinians, but talks are stalled.
Israeli Prime Minister Benjamin Netanyahu is meeting with President Joe Biden in Washington.
Netanyahu is hesitant to make concessions to the Palestinians, who are demanding a state of their own.
The Palestinians have been negot

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Pam

 Pam Or

 Pam Orton

 Pam Orton,

...

 being shaped by advancements in machine learning, natural language processing, and computer vision. However, as AI continues to evolve, we must consider the potential implications on the workforce, including job displacement and the need for workers to acquire new skills.
To prepare for the changing job market, educators and policymakers can focus on developing programs that emphasize skills such as creativity, critical thinking, and problem-solving. These skills are essential for workers to adapt to new technologies and remain relevant in the job market.
The following are some potential AI-related job opportunities that may emerge in the next decade:
1. AI Ethics Specialist
2. Data Scientist


 being shaped by advancements in machine learning, natural language processing, and computer vision. However, as AI continues to evolve, we must consider the potential implications on the workforce, including job displacement and the need for workers to acquire new skills.
To prepare for the changing job market, educators and policymakers can focus on developing programs that emphasize skills such as creativity, critical thinking, and problem-solving. These skills are essential for workers to adapt to new technologies and remain relevant in the job market.
The following are some potential AI-related job opportunities that may emerge in the next decade:
1. AI Ethics Specialist
2. Data Scientist
3

 being shaped by advancements in machine learning, natural language processing, and computer vision. However, as AI continues to evolve, we must consider the potential implications on the workforce, including job displacement and the need for workers to acquire new skills.
To prepare for the changing job market, educators and policymakers can focus on developing programs that emphasize skills such as creativity, critical thinking, and problem-solving. These skills are essential for workers to adapt to new technologies and remain relevant in the job market.
The following are some potential AI-related job opportunities that may emerge in the next decade:
1. AI Ethics Specialist
2. Data Scientist
3.

 being shaped by advancements in machine learning, natural language processing, and computer vision. However, as AI continues to evolve, we must consider the potential implications on the workforce, including job displacement and the need for workers to acquire new skills.
To prepare for the changing job market, educators and policymakers can focus on developing programs that emphasize skills such as creativity, critical thinking, and problem-solving. These skills are essential for workers to adapt to new technologies and remain relevant in the job market.
The following are some potential AI-related job opportunities that may emerge in the next decade:
1. AI Ethics Specialist
2. Data Scientist
3. AI

 being shaped by advancements in machine learning, natural language processing, and computer vision. However, as AI continues to evolve, we must consider the potential implications on the workforce, including job displacement and the need for workers to acquire new skills.
To prepare for the changing job market, educators and policymakers can focus on developing programs that emphasize skills such as creativity, critical thinking, and problem-solving. These skills are essential for workers to adapt to new technologies and remain relevant in the job market.
The following are some potential AI-related job opportunities that may emerge in the next decade:
1. AI Ethics Specialist
2. Data Scientist
3. AI Trainer

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Emily and I am a young, passionate baker from a small town in the countryside. I've been baking since I was a little girl, and I've always been fascinated by the magic of combining simple ingredients to create something truly special. In this blog, I'll be sharing my favourite recipes, tips, and stories from the kitchen. From classic cakes and cookies to more adventurous desserts, I hope you'll join me on this journey of discovery and indulgence.

**My Baking Style**

I like to think of my baking style as rustic-chic. I love using high-quality, locally-sourced ingredients and traditional techniques to create desserts that

Prompt: The capital of France is
Generated text:  facing a new urban transportation challenge: how to make public transportation more accessible and efficient for all citizens. The city of Paris has embarked on a comprehensive transportation plan to address the issues of congestion, pollution, and mobility for its 2.1 milli

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Bob

 Bob,

 Bob, and

 Bob, and I

...

 uncertain, but we can say with certainty that it will be shaped by the collective efforts of researchers, developers, policymakers, and users. In this section, we'll explore the current state of AI and its potential future directions.

## Key Challenges in AI Development

AI development is hindered by several challenges, including:

1.  **Bias and fairness**: AI systems can perpetuate and amplify existing social biases, leading to unfair outcomes. Ensuring fairness and transparency in AI decision-making is crucial.
2.  **Explainability and interpretability**: As AI systems become increasingly complex, it's difficult to understand how they

 uncertain, but we can say with certainty that it will be shaped by the collective efforts of researchers, developers, policymakers, and users. In this section, we'll explore the current state of AI and its potential future directions.

## Key Challenges in AI Development

AI development is hindered by several challenges, including:

1.  **Bias and fairness**: AI systems can perpetuate and amplify existing social biases, leading to unfair outcomes. Ensuring fairness and transparency in AI decision-making is crucial.
2.  **Explainability and interpretability**: As AI systems become increasingly complex, it's difficult to understand how they arrive

 uncertain, but we can say with certainty that it will be shaped by the collective efforts of researchers, developers, policymakers, and users. In this section, we'll explore the current state of AI and its potential future directions.

## Key Challenges in AI Development

AI development is hindered by several challenges, including:

1.  **Bias and fairness**: AI systems can perpetuate and amplify existing social biases, leading to unfair outcomes. Ensuring fairness and transparency in AI decision-making is crucial.
2.  **Explainability and interpretability**: As AI systems become increasingly complex, it's difficult to understand how they arrive at

 uncertain, but we can say with certainty that it will be shaped by the collective efforts of researchers, developers, policymakers, and users. In this section, we'll explore the current state of AI and its potential future directions.

## Key Challenges in AI Development

AI development is hindered by several challenges, including:

1.  **Bias and fairness**: AI systems can perpetuate and amplify existing social biases, leading to unfair outcomes. Ensuring fairness and transparency in AI decision-making is crucial.
2.  **Explainability and interpretability**: As AI systems become increasingly complex, it's difficult to understand how they arrive at their

 uncertain, but we can say with certainty that it will be shaped by the collective efforts of researchers, developers, policymakers, and users. In this section, we'll explore the current state of AI and its potential future directions.

## Key Challenges in AI Development

AI development is hindered by several challenges, including:

1.  **Bias and fairness**: AI systems can perpetuate and amplify existing social biases, leading to unfair outcomes. Ensuring fairness and transparency in AI decision-making is crucial.
2.  **Explainability and interpretability**: As AI systems become increasingly complex, it's difficult to understand how they arrive at their decisions

In [6]:
llm.shutdown()